In [ ]:
!pip install ultralytics pandas -q
import os
import yaml
import pandas as pd
import logging
from ultralytics import YOLO
from google.colab import drive
from IPython.display import display, clear_output

# --- 0. Instalación e Imports ---
clear_output()

# Desactiva los logs detallados de Ultralytics durante la validación
logging.getLogger('ultralytics').setLevel(logging.ERROR)
print("Bibliotecas instaladas y logging configurado.")

# --- 1. Montar Google Drive ---
try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Google Drive Montado.")
except Exception as e:
    print(f"❌ Error al montar Drive: {e}")

# --- 2. CONFIGURACIÓN PRINCIPAL ---

# Ruta al dataset unificado que mencionaste
DATASET_PATH = "/content/drive/MyDrive/Coffee_Unified_Seg_v1"
DATASET_YAML = os.path.join(DATASET_PATH, "data.yaml")

# Ruta donde se encuentran tus 5 archivos .pt
MODELS_BASE_PATH = "/content/drive/MyDrive/YOLOv8_Trained_Models/"

# Definimos la lista EXACTA de modelos a evaluar (sin 'type')
MODELS_TO_EVALUATE = [
    {'file': 'best_F1_yolov8x_img640_b-1.pt', 'imgsz': 640},
    {'file': 'best_F2_finetune_img1024_from_F1_yolov8x_img640_b-1.pt', 'imgsz': 1024},
    {'file': 'best_model_run_yolov8x-seg_img1024_b16_balanced.pt', 'imgsz': 1024},
    {'file': 'best_run_yolov8l_img640_b-1_stable_v2.pt', 'imgsz': 640},
]

# Batch size seguro para validación
SAFE_VALIDATION_BATCH = 16

print("--- Configuración Cargada ---")
print(f"Dataset YAML: {DATASET_YAML}")
print(f"Carpeta de Modelos: {MODELS_BASE_PATH}")
print("¡ADVERTENCIA! Se asumió 'imgsz=640' para el modelo 'train_M_bal1600...pt'.")

# --- 3. Verificar Dataset y Partición (Split) ---

EVAL_SPLIT = 'val' # Usar 'val' por defecto
try:
    with open(DATASET_YAML, 'r') as f:
        data_yaml = yaml.safe_load(f)

    # Usar 'test' si existe, si no, usar 'val'
    if 'test' in data_yaml and data_yaml['test']:
        EVAL_SPLIT = 'test'

    print(f"✅ Dataset YAML encontrado. Se usará la partición: '{EVAL_SPLIT}'")

except FileNotFoundError:
    print(f"❌ ERROR CRÍTICO: No se pudo encontrar el archivo {DATASET_YAML}")
    print("Asegúrate de que la ruta y el nombre del archivo son correctos.")
    raise

except Exception as e:
    print(f"❌ ERROR CRÍTICO al leer el YAML: {e}")
    raise

# --- 4. Bucle de Evaluación de Modelos ---

print(f"\nDespertando y listando la carpeta de Google Drive...")
!ls -lh "$MODELS_BASE_PATH"

print(f"\nIniciando evaluación de los {len(MODELS_TO_EVALUATE)} modelos específicos...")
print(f"ADVERTENCIA: Esto puede tardar MUCHO tiempo.")

all_results = []
# --- Variables para guardar el MEJOR modelo ---
best_mask_map = -1.0
best_model_metrics = None
best_model_name = ""

for model_info in MODELS_TO_EVALUATE:
    model_name = model_info['file']
    model_path = os.path.join(MODELS_BASE_PATH, model_name)
    val_imgsz = model_info['imgsz']

    print(f"\n--- Procesando: {model_name} ---")

    if not os.path.exists(model_path):
        print(f"... ❌ Omitiendo. No se encontró el archivo en: {model_path}")
        continue

    try:
        # Cargar modelo
        print(f"... Cargando modelo (imgsz: {val_imgsz}, batch: {SAFE_VALIDATION_BATCH})")
        model = YOLO(model_path)

        # Ejecutar validación
        print(f"... Ejecutando validación en '{EVAL_SPLIT}'...")
        metrics = model.val(
            data=DATASET_YAML,
            split=EVAL_SPLIT,
            imgsz=val_imgsz,
            batch=SAFE_VALIDATION_BATCH,
            verbose=False # No inundar la salida
        )

        # Preparar diccionario de resultados
        result_data = {
            'Modelo': model_name,
            'imgsz': val_imgsz
        }

        # Extraer métricas de Box y Mask
        try:
            result_data['Box mAP50-95'] = metrics.box.map
            result_data['Box mAP50'] = metrics.box.map50
        except Exception:
            result_data['Box mAP50-95'] = 0.0
            result_data['Box mAP50'] = 0.0

        try:
            result_data['Mask mAP50-95'] = metrics.seg.map
            result_data['Mask mAP50'] = metrics.seg.map50
        except Exception:
            result_data['Mask mAP50-95'] = 0.0
            result_data['Mask mAP50'] = 0.0

        all_results.append(result_data)

        # --- Comprobar si este es el mejor modelo hasta ahora ---
        current_mask_map = result_data.get('Mask mAP50-95', 0.0)
        if current_mask_map > best_mask_map:
            best_mask_map = current_mask_map
            best_model_metrics = metrics  # Guardamos el objeto COMPLETO de métricas
            best_model_name = model_name
            print(f"... ⭐ ¡Nuevo mejor modelo encontrado! (Mask mAP: {current_mask_map:.4f})")
        else:
            print(f"... ✅ Completado. Box mAP: {result_data['Box mAP50-95']:.4f}, Mask mAP: {result_data['Mask mAP50-95']:.4f}")

    except Exception as e:
        print(f"... ❌ ERROR al procesar este modelo ({model_name}): {e}")

# --- 5. Mostrar Tabla de Resultados Finales ---

print("\n\n--- 🏆 TABLA DE COMPARACIÓN FINAL 🏆 ---")

if not all_results:
    print("No se pudo validar ningún modelo.")
else:
    # Convertir la lista de resultados en un DataFrame de Pandas
    results_df = pd.DataFrame(all_results)

    # Ordenar por 'Mask mAP50-95'
    results_df_sorted = results_df.sort_values(by='Mask mAP50-95', ascending=False)

    # Formatear los números para mejor legibilidad
    for col in ['Box mAP50-95', 'Box mAP50', 'Mask mAP50-95', 'Mask mAP50']:
        if col in results_df_sorted.columns:
            results_df_sorted[col] = results_df_sorted[col].map('{:.4f}'.format)

    # Definir el orden de las columnas (como la tabla que pediste)
    final_cols = [
        'Modelo',
        'Mask mAP50-95', 'Mask mAP50',
        'Box mAP50-95', 'Box mAP50',
        'imgsz'
    ]

    # Filtrar solo a las columnas que existen
    final_cols = [c for c in final_cols if c in results_df_sorted.columns]

    print(f"Modelos ordenados por 'Mask mAP50-95' (de mejor a peor) usando la partición '{EVAL_SPLIT}' del dataset.")
    display(results_df_sorted[final_cols])


# --- 6.  Desglose por Clase súper detallado ---

print(f"\n\n--- 📊 MÉTRICAS DETALLADAS DEL MEJOR MODELO ---")
print(f"Mejor Modelo: {best_model_name}")

if best_model_metrics:
    try:
        print("\n--- Desglose por Clase (Métricas Box y Mask) ---")

        # Encabezado de la tabla (como la imagen)
        header = f"{'Clase':<15} | {'Box(P)':<10} {'Box(R)':<10} {'Box(mAP50)':<12} {'Box(mAP.5-.95)':<15} | {'Mask(P)':<10} {'Mask(R)':<10} {'Mask(mAP50)':<12} {'Mask(mAP.5-.95)':<15}"
        print(header)
        print("-" * len(header))

        # --- Extraer TODOS los arrays de métricas ---
        # Box
        box_p = best_model_metrics.box.p
        box_r = best_model_metrics.box.r
        box_ap50 = best_model_metrics.box.ap50
        box_maps = best_model_metrics.box.maps
        # Mask
        seg_p = best_model_metrics.seg.p
        seg_r = best_model_metrics.seg.r
        seg_ap50 = best_model_metrics.seg.ap50
        seg_maps = best_model_metrics.seg.maps

        # Imprimir métricas 'all' (promedio)
        all_line = f"{'all':<15} | {best_model_metrics.box.mp:<10.4f} {best_model_metrics.box.mr:<10.4f} {best_model_metrics.box.map50:<12.4f} {best_model_metrics.box.map:<15.4f} | {best_model_metrics.seg.mp:<10.4f} {best_model_metrics.seg.mr:<10.4f} {best_model_metrics.seg.map50:<12.4f} {best_model_metrics.seg.map:<15.4f}"
        print(all_line)

        # Imprimir métricas por clase
        for i, name in best_model_metrics.names.items():
            # Box
            b_p_val = box_p[i] if i < len(box_p) else 0.0
            b_r_val = box_r[i] if i < len(box_r) else 0.0
            b_ap50_val = box_ap50[i] if i < len(box_ap50) else 0.0
            b_map_val = box_maps[i] if i < len(box_maps) else 0.0
            # Mask
            s_p_val = seg_p[i] if i < len(seg_p) else 0.0
            s_r_val = seg_r[i] if i < len(seg_r) else 0.0
            s_ap50_val = seg_ap50[i] if i < len(seg_ap50) else 0.0
            s_map_val = seg_maps[i] if i < len(seg_maps) else 0.0

            # Formateamos la línea
            line = f"{name:<15} | {b_p_val:<10.4f} {b_r_val:<10.4f} {b_ap50_val:<12.4f} {b_map_val:<15.4f} | {s_p_val:<10.4f} {s_r_val:<10.4f} {s_ap50_val:<12.4f} {s_map_val:<15.4f}"
            print(line)

        print("\nNota: Las columnas 'Images' e 'Instances' son parte del log de validación en vivo (que se suprime con verbose=False) y no se guardan en el objeto de métricas final.")

    except Exception as e:
        print(f"  ❌ Error al extraer métricas detalladas: {e}")
else:
    print("No se pudo determinar el mejor modelo o no se guardaron sus métricas.")

Bibliotecas instaladas y logging configurado.
Mounted at /content/drive
✅ Google Drive Montado.
--- Configuración Cargada ---
Dataset YAML: /content/drive/MyDrive/Coffee_Unified_Seg_v1/data.yaml
Carpeta de Modelos: /content/drive/MyDrive/YOLOv8_Trained_Models/
¡ADVERTENCIA! Se asumió 'imgsz=640' para el modelo 'train_M_bal1600...pt'.
✅ Dataset YAML encontrado. Se usará la partición: 'test'

Despertando y listando la carpeta de Google Drive...
total 501M
-rw------- 1 root root 138M Jun 21 04:20 best_F1_yolov8x_img640_b-1.pt
-rw------- 1 root root 138M Jun 21 15:01 best_F2_finetune_img1024_from_F1_yolov8x_img640_b-1.pt
-rw------- 1 root root 138M Jun 20 23:36 best_model_run_yolov8x-seg_img1024_b16_balanced.pt
-rw------- 1 root root  89M Jun 21 02:20 best_run_yolov8l_img640_b-1_stable_v2.pt
drwx------ 2 root root 4.0K May 30 01:56 train_M_bal1600_e200_b32_lr0.01_colorFocus_Results

Iniciando evaluación de los 4 modelos específicos...
ADVERTENCIA: Esto puede tardar MUCHO tiempo.

--- Proce

,Modelo,Mask mAP50-95,Mask mAP50,Box mAP50-95,Box mAP50,imgsz
1,best_F2_finetune_img1024_from_F1_yolov8x_img64...,0.8555,0.9168,0.8756,0.9169,1024
3,best_run_yolov8l_img640_b-1_stable_v2.pt,0.8462,0.9260,0.8897,0.9260,640
0,best_F1_yolov8x_img640_b-1.pt,0.8281,0.9342,0.8928,0.9345,640
2,best_model_run_yolov8x-seg_img1024_b16_balance...,0.7727,0.8416,0.7964,0.8396,1024




--- 📊 MÉTRICAS DETALLADAS DEL MEJOR MODELO ---
Mejor Modelo: best_F2_finetune_img1024_from_F1_yolov8x_img640_b-1.pt

--- Desglose por Clase (Métricas Box y Mask) ---
Clase           | Box(P)     Box(R)     Box(mAP50)   Box(mAP.5-.95)  | Mask(P)    Mask(R)    Mask(mAP50)  Mask(mAP.5-.95)
-------------------------------------------------------------------------------------------------------------------------
all             | 0.9054     0.8643     0.9169       0.8756          | 0.9054     0.8643     0.9168       0.8555         
Inmaduro        | 0.9361     0.9729     0.9690       0.9172          | 0.9361     0.9729     0.9686       0.8867         
Pinton          | 0.9463     0.8685     0.9582       0.9253          | 0.9463     0.8685     0.9582       0.9009         
Maduro          | 0.9534     0.8700     0.9616       0.9406          | 0.9534     0.8700     0.9616       0.9115         
Sobremaduro     | 0.7658     0.8500     0.7862       0.7285          | 0.7658     0.8500     0.7862 